# Impl

In [1]:
#| default_exp app

In [2]:
#|export
import uvicorn
from dataclasses import dataclass

from starlette.responses import FileResponse, RedirectResponse, JSONResponse, HTMLResponse
from fastcore.utils import *
from fastcore.xml import *
from fasthtml import *

In [3]:
from pprint import pprint
from IPython import display

In [4]:
show_html(picocondlink)

In [5]:
set_pico_cls()

<IPython.core.display.Javascript object>

In [6]:
#|export
htmxscr = Script(
    src="https://unpkg.com/htmx.org@1.9.12", crossorigin="anonymous",
    integrity="sha384-ujb1lZYygJmzgSwoxRggbCHcjc0rB2XoQrxeTUQyRjrOnlCoYta87iKBWq3EsdM2")
mycss = Link(rel="stylesheet", href="picovars.css")

In [7]:
# debug=True, exception_handlers=exception_handlers

In [8]:
#|export
@dataclass
class TodoItem:
    title: str; id: int = -1; done: bool = False

TODO_LIST = [TodoItem(id=0, title="Start writing todo list", done=True),
             TodoItem(id=1, title="???", done=False),
             TodoItem(id=2, title="Profit", done=False)]

In [9]:
#|export
app = FastHTML()

reg_re_param("static", "ico|gif|jpg|jpeg|webm|css|js")
@app.get("/{fname:path}.{ext:static}")
async def image(fname:str, ext:str): return FileResponse(f'{fname}.{ext}')
@app.get("/static/{fname:path}")
async def static(fname:str): return FileResponse(f'static/{fname}')

In [10]:
#|export
id_curr = 'current-todo'
id_list = 'todo-list'
def tid(id): return f'todo-{id}'

In [11]:
#|export
@patch
def __xt__(self:TodoItem):
    show = AX(self.title, f'/todos/{self.id}', id_curr)
    edit = AX('edit',     f'/edit/{self.id}' , id_curr)
    dt = ' (done)' if self.done else ''
    return Li(show, dt, ' | ', edit, id=tid(self.id))

In [12]:
@patch
def _repr_html_(self:XT): return to_xml(self)

@patch
def _repr_html_(self:TodoItem): return self.__xt__()._repr_html_()

@patch
def md(self:XT): return display.Markdown(self._repr_markdown_())

@patch
def md(self:TodoItem): return self.__xt__().md()

In [13]:
todo = TODO_LIST[0]
todo

TodoItem(title='Start writing todo list', id=0, done=True)

In [14]:
todo.md()

```html
<li id="todo-0">
  <a href="#" hx-get="/todos/0" hx-target="#current-todo">
Start writing todo list
  </a>
 (done)
 | 
  <a href="#" hx-get="/edit/0" hx-target="#current-todo">
edit
  </a>
</li>

```

In [15]:
Ul(*TODO_LIST)

['ul',
 (TodoItem(title='Start writing todo list', id=0, done=True),
  TodoItem(title='???', id=1, done=False),
  TodoItem(title='Profit', id=2, done=False)),
 {}]

In [16]:
#|export
def mk_input(**kw): return Input(name="title", placeholder="New Todo", **kw)

In [17]:
inp = Group(mk_input(), Button("Add"))
add = Form(inp, id='add')
add

['form',
 (['fieldset',
   (['input', (), {'name': 'title', 'placeholder': 'New Todo'}],
    ['button', ('Add',), {}]),
   {'role': 'group'}],),
 {'id': 'add', 'name': 'add'}]

In [18]:
Card(Ul(*TODO_LIST, id=id_list),
     header=add, footer=Div(id=id_curr))

['article',
 (['header',
   (['form',
     (['fieldset',
       (['input', (), {'name': 'title', 'placeholder': 'New Todo'}],
        ['button', ('Add',), {}]),
       {'role': 'group'}],),
     {'id': 'add', 'name': 'add'}],),
   {}],
  ['ul',
   (TodoItem(title='Start writing todo list', id=0, done=True),
    TodoItem(title='???', id=1, done=False),
    TodoItem(title='Profit', id=2, done=False)),
   {'id': 'todo-list'}],
  ['footer', (['div', (), {'id': 'current-todo'}],), {}]),
 {}]

In [19]:
#|export
@app.get("/")
async def get_todos(req):
    inp = Group(mk_input(), Button("Add"))
    add = Form(inp, hx_post="/", target_id=id_list, hx_swap="beforeend")
    content = Card(
        Ul(*TODO_LIST, id=id_list),
        header=add, footer=Div(id=id_curr))
    return Html(
        Head(Title('TODO list'), htmxscr, picolink, mycss),
        Body(Main(H1('Todo list'), content, cls='container')))

In [20]:
#|export
@app.post("/")
async def add_item(todo:TodoItem):
    todo.id = len(TODO_LIST)+1
    TODO_LIST.append(todo)
    return todo, mk_input(hx_swap_oob='true')

In [21]:
#|export
def find_todo(id):
    try: return next(o for o in TODO_LIST if o.id==id)
    except: raise NotFoundException(f'Todo #{id}') from None

In [22]:
#|export
def clr_details(): return Div(hx_swap_oob='innerHTML', id=id_curr)

In [23]:
#|export
@app.get("/edit/{id}")
async def edit_item(id:int):
    form = Form(Group(Input(id="title"), Button("Save")),
                Hidden(id="id"), Checkbox(id="done", label='Done'),
                hx_put="/", target_id=tid(id), id="edit")
    return fill_form(form, find_todo(id))

In [24]:
#|export
@app.put("/")
async def update(todo: TodoItem):
    fill_dataclass(todo, find_todo(todo.id))
    return todo, clr_details()

In [25]:
#|export
@app.delete("/todos/{id}")
async def del_todo(id:int):
    TODO_LIST.remove(find_todo(id))
    return clr_details()

In [26]:
#|export
@app.get("/todos/{id}")
async def get_todo(id:int):
    todo = find_todo(id)
    btn = Button('delete', hx_delete=f'/todos/{todo.id}',
                 target_id=tid(todo.id), hx_swap="outerHTML")
    return Div(Div(todo.title), btn)

## export -

In [28]:
from nbdev.export import nb_export
nb_export('app.ipynb', '.')